In [6]:
!pip install lxml
!pip install shapely


In [7]:
import os
import ctypes

dll_path = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\openslide-bin-4.0.0.8-windows-x64\bin"
os.add_dll_directory(dll_path)  # Python 3.8 이상에서만 사용 가능

# 혹은 수동으로 로드 (fallback)
# ctypes.WinDLL(os.path.join(dll_path, "libopenslide-0.dll"))



<AddedDllDirectory('C:\\Users\\roobe\\PKG - HER2 tumor ROIs_v3\\openslide-bin-4.0.0.8-windows-x64\\bin')>

## 용량/데이터 확인

In [11]:
#용량 부족해서 bach 파일 삭제
import shutil
import os

patches_dir = r"C:\users\roobe\BACH\norm\train\patches"
# 폴더 내 모든 파일과 폴더 삭제
for filename in os.listdir(patches_dir):
    file_path = os.path.join(patches_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 하위 폴더 삭제
    except Exception as e:
        print(f"삭제 실패: {file_path}, 이유: {e}")


In [5]:

import shutil
import os

patches_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_HER2_cohort/val/1"
# 폴더 내 모든 파일과 폴더 삭제
for filename in os.listdir(patches_dir):
    file_path = os.path.join(patches_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # 파일 또는 링크 삭제
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # 하위 폴더 삭제
    except Exception as e:
        print(f"삭제 실패: {file_path}, 이유: {e}")

In [15]:
import os

files = os.listdir(r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\SVS")
o_files = [f for f in files if f.startswith('O') and f.endswith('.svs')]
s_files = [f for f in files if f.startswith('S') and f.endswith('.svs')]

print(f"O files: {len(o_files)}")
print(f"S files: {len(s_files)}")
#일단 이따 여쭤보께요...엑셀파일 안열리고 섦명에서 양성 36 음성 49라고 했는데 숫자가 이런데 저 s/o를 기준으로 라벨링을 ㅎ도 될까요

O files: 7
S files: 78


In [2]:
import pandas as pd

# 엑셀 파일 경로
file_path = r'C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\her2.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(file_path)

# 데이터프레임 출력
print(df)

      Patient     Responder  Response
0   O09-03495  nonresponder  positive
1   O10-12717  nonresponder  positive
2   O14-02301  nonresponder  positive
3   O16-11870     responder  positive
4   O16-18464  nonresponder  positive
..        ...           ...       ...
80  S18-31022     responder  positive
81  S18-32412  nonresponder  positive
82  S13-07627     responder  positive
83  S13-08586  nonresponder  positive
84  S18-15441     responder  positive

[85 rows x 3 columns]


## 패치 나누고 라벨 나누기

In [6]:
#파일에 0가들어가면 patches폴더의 야성, s는 음성 폴더에 저장하기 
#패치는 이미 다 나눠놨는데 나눌 때 쓴 코드가  날라가서 없어져버렸어요

import random

# 전제 설정 (필요한 경우 사용자에 맞게 경로 수정)
patch_dir = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches"
output_base = r"C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\patches"
metadata_path =r'C:\Users\roobe\PKG - HER2 tumor ROIs_v3\Yale_trastuzumab_response_cohort\her2.xlsx'

# 엑셀 데이터 불러오기
df = pd.read_excel(metadata_path)

# 환자 목록 분리
patients = df['Patient'].unique().tolist()  #한 환자에게서 나온 패치는 train/test중 하나의 폴더에만 전부 들어가 있어야함
random.seed(42)
random.shuffle(patients) #환자를 엑셀 리스트->파이썬 리스트로 만든 후 순서 섞기
split_idx = int(len(patients) * 0.8)
train_patients = set(patients[:split_idx])
test_patients = set(patients[split_idx:])

# 폴더 생성
for group in ['train', 'test']:
    for label in ['responder', 'nonresponder']: #train/test파일 안에 환자 분리할 파일 이름
        os.makedirs(os.path.join(output_base, group, label), exist_ok=True) #라벨 파일 경로 만들기

# 전체 패치 파일 목록
all_patches = [f for f in os.listdir(patch_dir) if f.endswith('.png')]

# 파일 이동
for patch in all_patches:
    patient_id = patch.split('_')[0] #패치파일이름에서 앞글자 가져와서 환자 이름 지정
    if patient_id not in df['Patient'].values:
        continue

    label = df.loc[df['Patient'] == patient_id, 'Responder'].values[0] #파일 앞에 엑셀시트 환자이름과 같은지 확인/엑셀에서 'Responder' 부분 확인
    group = 'train' if patient_id in train_patients else 'test' #앞에 환자 그루 지정하 때 train 에 있었다면 거기에 넣기

    src_path = os.path.join(patch_dir, patch)
    dst_path = os.path.join(output_base, group, label, patch)

    shutil.move(src_path, dst_path)  # 복사 → 이동으로 변경

print("패치 이동 및 정리 완료!")

#실행 전에 지울꺼 지워야함 


패치 이동 및 정리 완료!


In [ ]:
#이거 먼저 코드 돌린 다음에 이 패치 파일 지우고 암조직만 묶어서 다시 패치 나눠야 용량이 가능할것같아용